In [1]:
import random
import numpy as np
import itertools

from cmath import *

# Simulator Program

##### ground state

In [2]:
def get_ground_state(num_qubits):
    # return vector of size 2**num_qubits with all zeroes except first element which is 1
    ground_state = np.zeros(2**num_qubits)
    ground_state[0] = 1
    return ground_state

#### get_operator
This function works on any N control bits (https://github.com/quantastica/qosf-mentorship/blob/master/qosf-simulator-task-additional-info.pdf)

In [3]:
def get_operator_nqubit(total_qubits, gate_unitary, target_qubits):
    
    control_bit = target_qubits[:-1]
    target_bit = target_qubits[-1]
    sub_matrix  = []
    combination =  list(map(list, itertools.product([0, 1], repeat=len(control_bit))))

    p_0x0_1x1 = [P0x0,P1x1]
    for q in combination[:-1]:
        matrix = 1
        pointer = 0
        for i in range(total_qubits):
            if i in control_bit:
                matrix = np.kron(matrix ,p_0x0_1x1[q[pointer]])
                pointer+=1
            else:
                matrix = np.kron(matrix,I)                
        sub_matrix.append(matrix)
        
        
    matrix = 1
    for i in range(total_qubits):
        if i == target_bit:
            matrix = np.kron(matrix ,gate_unitary)
        
        elif i in control_bit:
            matrix = np.kron(matrix ,P1x1)

        else:
            matrix = np.kron(matrix,I)      
    sub_matrix.append(matrix)
    operator = np.sum(sub_matrix, axis = 0)
    return operator

# run_program

In [4]:
def run_program(initial_state, program):
    total_qubits = int(np.log2(len(initial_state)))
    unitary = np.identity(len(initial_state))
    # read program, and for each gate:
    #   - calculate matrix operator
    
    for layer in program:
        target_qubits = layer['target']
        gate_unitary = gate_name_unitary[ layer['gate']]
        operator = get_operator_nqubit(total_qubits, gate_unitary, target_qubits)

        unitary = np.dot(unitary,operator)
            
            
    #   - multiply state with operator
    final_state = np.dot(initial_state,unitary)
    
    # return final state
    return final_state

# Measure

In [5]:
def measure_all(state_vector):
    # choose element from state_vector using weighted random and return it's index
    position = 0
    cumulative_sum =0
    random_result = random.random()
    for i in state_vector:
        prob =abs(i)**2
        cumulative_sum+=prob
        if cumulative_sum <= random_result:
            position +=1
        else:
            return position

# Get counts

In [6]:
def get_counts(state_vector, num_shots):
    # simply execute measure_all in a loop num_shots times and
    # return object with statistics in following form:
    #   {
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      ...
    #   }
    # (only for elements which occoured - returned from measure_all)
    
    num_qubits = int(np.log2(len(state_vector)))
    combination = list(map(list, itertools.product([0, 1], repeat=num_qubits)))
    
    count = {}
    element = []
    for state in combination:
        string = "".join([str(i) for i in state])    
        count[string] = 0
        element.append(string)
    
    for experiment in range(num_shots):
        result = measure_all(state_vector)
        count[element[result]] +=1
        
    return count

### quantum gates and global variable

In [7]:
# Define projection operator |0><0|

P0x0 = np.array([
[1, 0],
[0, 0]
])

# Define projection operator |1><1|

P1x1 = np.array([
[0, 0],
[0, 1]
])



X = np.array([
[0, 1],
[1, 0]
])

H =0.5**0.5*np.array([
    [1, 1],
    [1,-1]
])

gate_name_unitary ={
    'h':H,
    'x':X,
    'cx':X,
    'ccx':X,
}

I = np.identity(2)

# run example

In [8]:
my_circuit = [
{ "gate": "h", "target": [0] }, 
{ "gate": "cx", "target": [0, 1] }
]

In [9]:
my_qpu = get_ground_state(2)
print(my_qpu)

[1. 0. 0. 0.]


In [10]:
final_state = run_program(my_qpu, my_circuit)
print(final_state)

[0.70710678 0.         0.         0.70710678]


In [11]:
get_counts(final_state, num_shots=1000)

{'00': 501, '01': 0, '10': 0, '11': 499}

# Parametric gates

#### define gates

In [12]:
def U1(params):
    theta = params['theta']
    return np.array([
        [1 , 0],
        [0, exp(1.j *lam )]
        
    ])

def U2(params):
    theta = params['theta']
    phi = params['phi']
    return 0.5**0.5*np.array([
        [1, -exp(1.j*lam)],
        [exp(1.j*phi), exp(1.j*(phi+lam))]
    ])

def U3(params):
    theta = params['theta']
    phi = params['phi']
    lam = params['lambda']
    return np.array([
        [cos(theta/2),  -exp(1.j*lam)*sin(theta/2)],
        [exp(1.j*phi)*sin(theta/2),  exp(1.j*(phi+lam))*cos(theta/2)]
    ])

param_gates ={
    'u1':U1,
    'u2':U2,
    'u3':U3
}

####  change run_program()

add condition for parametric gates

In [13]:
def run_program(initial_state, program):
    total_qubits = int(np.log2(len(initial_state)))
    unitary = np.identity(len(initial_state))
    # read program, and for each gate:
    #   - calculate matrix operator
    
    for layer in program:
        target_qubits = layer['target']
        
        # check if user use defined gate
        if layer['gate'] in gate_name_unitary:
            gate_unitary = gate_name_unitary[ layer['gate']]
            params = None
            
        #### add parametric gates ####
        elif layer['gate'] in param_gates:
            params = layer['params']
            gate_function = param_gates[ layer['gate']]
            gate_unitary = gate_function(params)
            
        else:
            gate_unitary = layer['gate']
            
        operator = get_operator_nqubit(total_qubits, gate_unitary, target_qubits)
        unitary = np.dot(unitary,operator)
            
            
    #   - multiply state with operator
    final_state = np.dot(initial_state,unitary)
    
    # return final state
    return final_state

In [14]:
my_qpu = get_ground_state(2)
print(my_qpu)

[1. 0. 0. 0.]


In [15]:
my_circuit = [
{ "gate": "h", "target": [0] }, 
{ "gate": "u3", "params": { "theta": 3.1415, "phi": 1.5708, "lambda": -3.1415 }, "target": [0] }
]

In [16]:
final_state = run_program(my_qpu, my_circuit)
print(final_state)

[3.01606426e-05+7.07106780e-01j 0.00000000e+00+0.00000000e+00j
 7.07106781e-01+3.27579908e-05j 0.00000000e+00+0.00000000e+00j]


In [17]:
get_counts(final_state, num_shots=1000)

{'00': 473, '01': 0, '10': 527, '11': 0}

# Allow running variational quantum algorithms

#####  change run_program()

edit for variational parameter

In [18]:
from copy import deepcopy

In [19]:
def run_program(initial_state, init_program, variational_param = None):
    total_qubits = int(np.log2(len(initial_state)))
    unitary = np.identity(len(initial_state))
    # read program, and for each gate:
    #   - calculate matrix operator
    
    
    #avoid changing values in the reference
    program = deepcopy(init_program)
    
    for layer in program:
        target_qubits = layer['target']
        
        # check if user use defined gate
        if layer['gate'] in gate_name_unitary:
            gate_unitary = gate_name_unitary[ layer['gate']]
            params = None
        # add param gates
        elif layer['gate'] in param_gates:
            params = layer['params']
            
            ### add variational_param
            if variational_param != None:
                for param_i in params:
                    if params[param_i] in variational_param:
                         params[param_i] = variational_param[params[param_i]]
            
            gate_function = param_gates[ layer['gate']]
            gate_unitary = gate_function(params)
            
        else:
            gate_unitary = layer['gate']
            
        operator = get_operator_nqubit(total_qubits, gate_unitary, target_qubits)
        unitary = np.dot(unitary,operator)
            
            
    #   - multiply state with operator
    final_state = np.dot(initial_state,unitary)
    
    # return final state
    return final_state

In [20]:
my_circuit = [
  { "gate": "u3", "params": { "theta": "global_1", "phi": "global_2", "lambda": -3.1415 }, "target": [0] }
]
params = np.array([3.1415, 1.5708])
final_state = run_program(my_qpu, my_circuit, { "global_1": params[0], "global_2": params[1] })

In [21]:
final_state

array([4.63267949e-05+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
       9.99999995e-01+9.26535896e-05j, 0.00000000e+00+0.00000000e+00j])

# test variational circuit

In [22]:
from scipy.optimize import minimize

In [23]:
my_qpu = get_ground_state(2)
print(my_qpu)

[1. 0. 0. 0.]


In [24]:
my_circuit = [
{ "gate": "u3", "target": [0], "params": { "theta": "global_1", "phi": 0, "lambda": 0} },
{ "gate": "u3", "target": [1], "params": { "theta": "global_2", "phi": 0, "lambda": 0 } }
]

### objective function

The objectives function design for state [0,0,0,1]

In [25]:
def objective_function(params):
    final_state = run_program(my_qpu, my_circuit, { "global_1": params[0], "global_2": params[1]})

    counts = get_counts(final_state, 1000)

    # i want state [0,0,0,1]
    cost =  (1-counts['11']/1000)**2
    return cost

### optimize

In [26]:
#init_values
init_params = np.random.rand(2)
print('init params', init_params)

init params [0.712749   0.56260591]


In [27]:
first_state = run_program(my_qpu, my_circuit, { "global_1": init_params[0], "global_2": init_params[1]})
counts = get_counts(first_state, 1000)
print('before optimize: counts ', counts)

before optimize: counts  {'00': 819, '01': 65, '10': 108, '11': 8}


In [28]:
minimum = minimize(objective_function, init_params, method="Powell", tol=1e-6)

In [29]:
minimum

   direc: array([[1., 0.],
       [0., 1.]])
     fun: array(0.)
 message: 'Optimization terminated successfully.'
    nfev: 184
     nit: 3
  status: 0
 success: True
       x: array([3.12651439, 3.24425873])

#### evaluate the variational circuit

In [30]:
optimized_param = minimum['x']
final_cost = objective_function(optimized_param)
print('final_cost', final_cost)

final_cost 0.0


In [31]:
final_state = run_program(my_qpu, my_circuit, { "global_1": optimized_param[0], "global_2": optimized_param[1]})

In [32]:
counts = get_counts(final_state, 1000)
print('counts ', counts)

counts  {'00': 0, '01': 0, '10': 2, '11': 998}
